In [38]:
import pandas as pd
import numpy as np
import json
import string
from itertools import groupby 

In [31]:
with open('../Files/English_dictionary.json', 'r') as f:
    dic = json.load(f)

In [34]:
total= dict.fromkeys(string.ascii_lowercase, 0)
total["-"]=0
total

{'a': 0,
 'b': 0,
 'c': 0,
 'd': 0,
 'e': 0,
 'f': 0,
 'g': 0,
 'h': 0,
 'i': 0,
 'j': 0,
 'k': 0,
 'l': 0,
 'm': 0,
 'n': 0,
 'o': 0,
 'p': 0,
 'q': 0,
 'r': 0,
 's': 0,
 't': 0,
 'u': 0,
 'v': 0,
 'w': 0,
 'x': 0,
 'y': 0,
 'z': 0,
 '-': 0}

# Calculate probability given 2 or more letters

In [39]:
le_no_word=["a", "b", 'g']

In [40]:
dic = [ele for ele in dic if all(ch not in ele for ch in le_no_word)] 

In [41]:
letters=["h", "o"]

In [42]:
dic = [ele for ele in dic if all(ch in ele for ch in letters)] 

In [43]:
for i in dic:
    for x in i:
        total[x]+=1

In [48]:
vowels=["a","e", "i", "o", "u"]

In [49]:
remove= list(set(vowels+list(letters)+le_no_word))
total= {k:v for k,v in total.items() if k not in remove}
total

{'c': 7398,
 'd': 3825,
 'f': 1004,
 'j': 69,
 'k': 894,
 'l': 5513,
 'm': 4548,
 'n': 6449,
 'p': 7248,
 'q': 69,
 'r': 10013,
 's': 11075,
 't': 9295,
 'v': 555,
 'w': 1156,
 'x': 375,
 'y': 4504,
 'z': 516,
 '-': 0}

In [50]:
df = pd.DataFrame.from_dict(total, orient ='index', columns=["total"])
df.index.name = "letter"
df.sort_values(by=['total'], ascending=False, inplace=True)
df.head()

,total
letter,
s,11075
r,10013
t,9295
c,7398
p,7248


In [51]:
df['per'] = (df.total * 100 / df.total.sum()).round(2).astype(str) + '%'
df.head()

,total,per
letter,,
s,11075,14.86%
r,10013,13.44%
t,9295,12.48%
c,7398,9.93%
p,7248,9.73%


In [11]:
#NoVowel

def next_letter(*letter):
    
    #Create df with letters as an index
    df_words= df.loc[letter]
    #Get new row with total letters appears
    total = df_words.sum()
    total.name = 'total'
    # Assign sum of all rows of DataFrame as a new Row
    df_words = df_words.append(total.transpose())
    vowels=["a","e", "i", "o", "u"]
    #Delete vowels and letters from columns and rows
    remove= list(set(vowels+list(*letter)))
    df_words.drop(*letter, axis=0, inplace=True)
    df_words.drop(remove, axis=1, inplace=True)
    #New column with percentajes for each vowel
    df_words= df_words.T[["total"]]
    df_words['per'] = (df_words.total * 100 / df_words.total.sum()).round(4)
    df_words.sort_values(by="total", ascending=False, inplace= True)
    
    #Get recommended letter
    final_word= df_words.total.idxmax()
    print(df_words.head())
    if (df_words.iloc[0].per - df_words.iloc[1].per) < 1:
        return f'You should choose {final_word} with {df_words.loc[final_word].per.round(3)}% appear or {df_words.iloc[1].name} with {df_words.loc[df_words.iloc[1].name].per.round(3)}% appear'
    else:
        return f'You should choose {final_word} with a {df_words.loc[final_word].per.round(3)}% appear'

In [12]:
letters=["d", "c"]
next_letter(letters)

    total      per
n  159372  14.7008
r  151901  14.0117
s  138365  12.7631
t  135365  12.4864
l  116969  10.7895


'You should choose n with 14.701% appear or r with 14.012% appear'

In [13]:
#Vowel

def next_vowel(*vowel):
    
    #Create df with letters as an index
    df_vowel= df.loc[vowel]
    print(vowel)
    #Get new row with total letters appears    
    total = df_vowel.sum()
    total.name = 'total'
    # Assign sum of all rows of DataFrame as a new Row
    vowels=["a","e", "i", "o", "u"]
    vowels= [i for i in vowels if i not in list(*vowel)]
    print(vowels)
    df_vowel = df_vowel.append(total.transpose())
    df_vowel = df_vowel[[*vowels]].T[["total"]]
    #New column with percentajes for each vowel
    df_vowel['per'] = (df_vowel.total * 100 / df_vowel.total.sum()).round(4)
    df_vowel.sort_values(by="total", ascending=False, inplace= True)
    
    #Get recommended vowel
    final_vowel= df_vowel.total.idxmax()
    print(df_vowel.head())
    if (df_vowel.iloc[0].per - df_vowel.iloc[1].per) < 1:
        return f'You should choose {final_vowel} with {df_vowel.loc[final_vowel].per.round(3)}% appear or {df_vowel.iloc[1].name} with {df_vowel.loc[df_vowel.iloc[1].name].per.round(3)}% appear'
    else:
        return f'You should choose {final_vowel} with a {df_vowel.loc[final_vowel].per.round(3)}% appear'

In [14]:
letters=["d", "c", "e", "o"]
next_vowel(letters)

(['d', 'c', 'e', 'o'],)
['a', 'i', 'u']
    total      per
i  567603  43.4227
a  507824  38.8495
u  231730  17.7278


'You should choose i with a 43.423% appear'

# Calcultate probability with empty word but we have some letters in the panel

In [15]:
with open('../Files/English_dictionary.json', 'r') as f:
    dic = json.load(f)

In [16]:
def empty_consonant(dic, letters):
    dic=list(dic.keys())
    total= dict.fromkeys(string.ascii_lowercase, 0)
    total["-"]=0
    dic = [ele for ele in dic if all(ch not in ele for ch in letters)] 
    for i in dic:
        for x in i:
            total[x]+=1
    df = pd.DataFrame.from_dict(total, orient ='index', columns=["total"])
    df.index.name = "letter"
    df.sort_values(by=['total'], ascending=False, inplace=True)
    vowels=["a","e", "i", "o", "u"]
    df.drop(vowels, axis=0, inplace=True)
    df['per'] = (df.total * 100 / df.total.sum()).round(2)
    final= df.total.idxmax()
    if (df.iloc[0].per - df.iloc[1].per) < 1:
        return f'You should choose {final} with {df.loc[final].per.round(3)}% appear or {df.iloc[1].name} with {df.loc[df.iloc[1].name].per.round(3)}% appear'
    else:
        return f'You should choose {final} with a {df.loc[final].per.round(3)}% appear'

In [17]:
letters=["a", "b", 'g']
empty_consonant(dic, letters)

'You should choose s with a 14.98% appear'

In [18]:
def empty_vowel(dic, vowel):
    dic=list(dic.keys())
    total= dict.fromkeys(string.ascii_lowercase, 0)
    total["-"]=0
    dic = [ele for ele in dic if all(ch not in ele for ch in vowel)] 
    for i in dic:
        for x in i:
            total[x]+=1
    vowels=["a","e", "i", "o", "u"]
    df = pd.DataFrame.from_dict(total, orient ='index', columns=["total"]).loc[vowels]
    df.index.name = "letter"
    df.sort_values(by=['total'], ascending=False, inplace=True)
    df['per'] = (df.total * 100 / df.total.sum()).round(2)
    print(df)
    final= df.total.idxmax()
    if (df.iloc[0].per - df.iloc[1].per) < 1:
        return f'You should choose {final} with {df.loc[final].per.round(3)}% appear or {df.iloc[1].name} with {df.loc[df.iloc[1].name].per.round(3)}% appear'
    else:
        return f'You should choose {final} with a {df.loc[final].per.round(3)}% appear'

In [19]:
letters=["a", "b", 'g']
vowels=["a","e", "i", "o", "u"]
empty_vowel(dic, letters)

         total    per
letter               
e       124566  37.66
i        82648  24.99
o        81574  24.66
u        41994  12.70
a            0   0.00


'You should choose e with a 37.66% appear'

In [20]:
total= dict.fromkeys(string.ascii_lowercase, 0)
total["-"]=0
total

{'a': 0,
 'b': 0,
 'c': 0,
 'd': 0,
 'e': 0,
 'f': 0,
 'g': 0,
 'h': 0,
 'i': 0,
 'j': 0,
 'k': 0,
 'l': 0,
 'm': 0,
 'n': 0,
 'o': 0,
 'p': 0,
 'q': 0,
 'r': 0,
 's': 0,
 't': 0,
 'u': 0,
 'v': 0,
 'w': 0,
 'x': 0,
 'y': 0,
 'z': 0,
 '-': 0}

In [21]:
letters=["a", "b", 'g']

In [22]:
dic = [ele for ele in dic if all(ch not in ele for ch in letters)] 

In [23]:
for i in dic:
    for x in i:
        total[x]+=1

In [24]:
total

{'a': 0,
 'b': 0,
 'c': 42337,
 'd': 34772,
 'e': 124566,
 'f': 14676,
 'g': 0,
 'h': 25327,
 'i': 82648,
 'j': 1674,
 'k': 9484,
 'l': 42658,
 'm': 27734,
 'n': 59009,
 'o': 81574,
 'p': 35908,
 'q': 1826,
 'r': 66986,
 's': 82394,
 't': 59785,
 'u': 41994,
 'v': 10980,
 'w': 7801,
 'x': 3479,
 'y': 19479,
 'z': 3547,
 '-': 0}

In [25]:
vowels=["a","e", "i", "o", "u"]

In [26]:
df = pd.DataFrame.from_dict(total, orient ='index', columns=["total"])
df

,total
a,0
b,0
c,42337
d,34772
e,124566
f,14676
g,0
h,25327
i,82648
j,1674


In [27]:
df = pd.DataFrame.from_dict(total, orient ='index', columns=["total"])
df.index.name = "letter"
df.sort_values(by=['total'], ascending=False, inplace=True)

In [28]:
df

,total
letter,
e,124566
i,82648
s,82394
o,81574
r,66986
t,59785
n,59009
l,42658
c,42337


In [29]:
df['per'] = (df.total * 100 / df.total.sum()).round(2).astype(str) + '%'
df.head()

,total,per
letter,,
e,124566,14.14%
i,82648,9.39%
s,82394,9.36%
o,81574,9.26%
r,66986,7.61%
